# Recency Effects in Pairwise Comparisons 
# during Peer Assessment of Oral Presentations
### Accepted for publication in Assessment and Evaluation in Higher Education
#### Authors: Erez Yaakobi, Idit Shalev, Lihi Dery
#### Code, data processing, statistical analysis, and figures: Lihi Dery
#### Contact: lihid@ariel.ac.il

In [18]:
import pandas as pd
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import statsmodels.formula.api as smf
from IPython.display import display

## Read & Combine data

In [20]:
def read_case_study(case_study):
    url = f"https://api.github.com/repos/nlihin/R2R-analysis/contents/data/{case_study}/recency_bias"
    response = requests.get(url)
    files = response.json()

    csv_url = next(f["download_url"] for f in files if f["name"].endswith(".csv"))
    return pd.read_csv(csv_url)


df1 = read_case_study("case_study_1")
df2 = read_case_study("case_study_2")

In [ ]:
df2 = df2[['user_id', 'score', 'lag', 'session_id', 'primacy', 'recency', 'total_comparisons', 'norm_primacy', 'norm_recency']]

In [ ]:
all_df = pd.concat([df1, df2], ignore_index=True)
all_df

In [ ]:
session_stats = (
    all_df
    .groupby(['session_id'])['user_id']
    .nunique()
    .reset_index(name='n_evaluators')
)

session_stats

## Analysis

In [ ]:
all_df1 = all_df

### Table 1 - Data collection statistics

In [ ]:
all_df1.groupby('session_id')['user_id'].nunique()

In [ ]:
all_df1['user_id'].nunique()

In [ ]:
all_df1["total_comparisons"].sum()

In [ ]:
5364/365

### Table 2

In [ ]:
tbl2_df = all_df1[['norm_recency', 'score', 'lag']] 

means = tbl2_df.mean().round(2)
stds = tbl2_df.std().round(2)

corr = tbl2_df.corr().round(3)

corr

In [ ]:
pvals = pd.DataFrame(index=tbl2_df.columns, columns=tbl2_df.columns)

for i in tbl2_df.columns:
    for j in tbl2_df.columns:
        _, p = pearsonr(tbl2_df[i], tbl2_df[j])
        pvals.loc[i, j] = round(p, 4)

pvals

In [ ]:
table_rows = []
variables = corr.columns.tolist()

for i, var in enumerate(variables):
    row = {
        'Predictor': f"{i+1}. {var}",
        'M': means[var],
        'SD': stds[var],
    }
    for j in range(len(variables)):
        if j < i:
            r = corr.iloc[i, j]
            p = float(pvals.iloc[i, j])
            star = "*" if p < 0.05 else ""
            row[str(j+1)] = f"{r:.3f}{star}"
        elif j == i:
            row[str(j+1)] = "-"
        else:
            row[str(j+1)] = ""
    table_rows.append(row)

formatted_table = pd.DataFrame(table_rows)

formatted_table

### Table 3 - Mixed effects 

In [ ]:
df =  all_df

agg_df = (
    df.groupby(["user_id", "score"])
      .agg(
          norm_recency=("recency", "mean"),
          mean_lag=("lag", "mean"),
          n_comparisons=("recency", "size")
      )
      .reset_index()
)

# Linear mixed-effects model
lmm = smf.mixedlm(
    "norm_recency ~ score + mean_lag",
    data=agg_df,
    groups=agg_df["user_id"]
)

lmm_result = lmm.fit()

print(lmm_result.summary())

## Figure (not included in the paper)

In [ ]:
def create_barplot(norm_df, scores, figsize=(4, 3)):
    cpal = "BrBG"
    figs = {}
    for score in scores:
        norm_df_score = norm_df[norm_df['score'] == score].melt(
            id_vars='lag', 
            value_vars=['norm_primacy', 'norm_recency'], 
            var_name='metric', 
            value_name='value'
        )
        
        norm_df_score['metric'] = norm_df_score['metric'].replace({
            'norm_primacy': 'primacy', 
            'norm_recency': 'recency'
        })
        
        fig, ax = plt.subplots(figsize=(4, 3))
        sns.barplot(data=norm_df_score, x='lag', y='value', hue='metric', ax=ax, palette=cpal)
        ax.legend(title='')
        figs[score] = fig  # Store the figure keyed by score
        plt.close(fig)

    return figs

In [ ]:
scores = [5, 4]
figs = create_barplot(all_df1[all_df1.lag<10], scores)  

In [ ]:
for score in scores:
    print ("Score: " ,score)
    display(figs[score])